<a href="https://colab.research.google.com/github/lrssv/ComputacaoEvolutiva/blob/Part-2/Initial_population.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model for classifying the victim's gender using an evolutionary algorithm to select features.

## - Initial Population

This code has all steps for the initial population. Where:

* Imports: the necessary libraries
* Scripts: imports the encoded dataset and create the initial population with 100 individuals, where each individual has 18 features, the first is the model and all others correspond for the features for the dataset
* Setups: results and export.

##Imports

In [44]:
import pandas as pd
import numpy as np
import random
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from pandas import read_csv 

## Scripts

In [19]:
def database(url):
  df_encoded = pd.read_csv(url)
  df_encoded = df_encoded.drop(columns=['Unnamed: 0'])

  return df_encoded

In [47]:
def population(ind_size,pop_size,models,ms,pc):
  ind = []
  pop = []
  probs = []

  for p in range(pop_size):
    probs.append(ms)
    probs.append(pc)

    ind = [random.choice([True,False]) for i in range(ind_size)] #features
    ind[0] = random.choice(range(0,len(models))) #model
    ind[1] = probs #probabilities
    pop.append(ind)
    ind = []
    probs = []

  print("-------------------------- Initial Population --------------------------")
  print(pd.DataFrame(pop, columns=['model','probabilities', 'feature 1', 'feature 2', 'feature 3','feature 4',
                                   'feature 5', 'feature 6', 'feature 7','feature 8', 'feature 9',
                                   'feature 10', 'feature 11','feature 12', 'feature 13', 'feature 14',
                                   'feature 15','feature 16', 'feature 17']))
  return pop

## Instances

In [21]:
df_encoded = database('/content/drive/My Drive/Data Files/df_model_per_encoded')

In [22]:
models = [DecisionTreeClassifier(max_depth=10),DecisionTreeClassifier(max_depth=3),DecisionTreeClassifier(max_depth=14),
          RandomForestClassifier(max_depth=10),RandomForestClassifier(max_depth=14)]

In [37]:
#Default
individual_size = 19
population_size = 100
mutation_strength = 1/(individual_size-2) 
prob_crossover = 0.8

In [48]:
initial_population = population(individual_size,population_size,models,mutation_strength,prob_crossover)

-------------------------- Initial Population --------------------------
    model                probabilities  ...  feature 16  feature 17
0       2  [0.058823529411764705, 0.8]  ...       False        True
1       0  [0.058823529411764705, 0.8]  ...        True       False
2       2  [0.058823529411764705, 0.8]  ...        True       False
3       4  [0.058823529411764705, 0.8]  ...       False       False
4       3  [0.058823529411764705, 0.8]  ...       False        True
..    ...                          ...  ...         ...         ...
95      0  [0.058823529411764705, 0.8]  ...       False       False
96      1  [0.058823529411764705, 0.8]  ...       False       False
97      1  [0.058823529411764705, 0.8]  ...        True        True
98      2  [0.058823529411764705, 0.8]  ...        True       False
99      3  [0.058823529411764705, 0.8]  ...        True       False

[100 rows x 19 columns]


In [49]:
#export population
with open("/content/drive/My Drive/Data Files/initial_population.txt", "wb") as fp: 
  pickle.dump(initial_population, fp)